In [8]:
import pymc3 as pm
import src.utils as utils
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import theano.tensor as tt
import theano 
import seaborn as sns
import pandas as pd
from sklearn.metrics import mean_squared_error

In [9]:
short_metrics_p, long_metrics_p = utils.read_data(shift=True)
short_metrics = short_metrics_p[:, :, 0]
long_metrics = long_metrics_p[:, :, 0]

target_metric_p = long_metrics_p[:, 0, :]   # <--- here you can choose target (0, 1, 2, 3)
target_metric = target_metric_p[:, 0]

In [10]:
sm_train, sm_test, lm_train, lm_test = train_test_split(short_metrics, target_metric)
train_size = len(sm_train)
test_size = len(sm_test)

In [15]:
x_0 = theano.shared(sm_train[:, 0])
x_1 = theano.shared(sm_train[:, 1])
x_2 = theano.shared(sm_train[:, 2])
x_3 = theano.shared(sm_train[:, 3])
y = theano.shared(lm_train)

with pm.Model() as test_model:
    sigma_2 = pm.HalfCauchy('sigma_2', 2)
    coef_2 = pm.Normal('coef_2', mu=0, sigma=1, shape=3)
    mu_2 = coef_2[0] + coef_2[1] * x_0 + coef_2[2] * x_1
    eps_2 = pm.Normal('eps_2', mu=0, sigma=sigma_2)
    xm_2 = pm.Deterministic('xm_2', mu_2 + eps_2)
    
    sigma_y = pm.HalfCauchy('sigma_y', 2)
    coef_y = pm.Normal('coef_y', mu=0, sigma=1, shape=3)
    mu_2 = coef_y[0] + coef_y[1] * xm_2 + coef_y[2] * x_3
    y = pm.Normal('Y', mu=mu_2, sigma=sigma_y, observed=y)
    
    approx = pm.fit(30000)
    # trace = pm.sample()

Average Loss = -158.41: 100%|██████████| 30000/30000 [00:14<00:00, 2078.40it/s] 
Finished [100%]: Average Loss = -158.4


In [16]:
x_0.set_value(sm_test[:, 0])
x_1.set_value(sm_test[:, 1])
x_2.set_value(sm_test[:, 2])
x_3.set_value(sm_test[:, 3])

In [ ]:
# MCMC error
ppc = pm.sample_posterior_predictive(trace, model=test_model, samples=1000)
pred_mcmc = ppc['Y'].mean(axis=0)
mse_mcmc = mean_squared_error(lm_test, pred_mcmc)
print(mse_mcmc)

In [17]:
# ADVI error
ppc = pm.sample_posterior_predictive(approx.sample(100000), model=test_model, samples=1000)
pred_advi = ppc['Y'].mean(axis=0)
mse_advi = mean_squared_error(lm_test, pred_advi)
print(mse_advi)

100%|██████████| 1000/1000 [00:16<00:00, 60.95it/s]


0.01110024886362027


In [18]:
x = theano.shared(sm_train)
y = theano.shared(lm_train)

linreg_model = pm.Model()
with linreg_model:
    sigma = pm.HalfCauchy('Sigma', beta=2)
    intercept = pm.Normal('Intercept', mu=0, sigma=1)
    coef = pm.Normal('Coef', mu=0, sigma=1, shape=17)
    ym = pm.Normal('Y', mu=intercept + pm.math.dot(x, coef), sigma=sigma, observed=y)
    # trace = pm.sample(cores=3, init='nuts')
    approx = pm.fit(50000)


Average Loss = -294.24: 100%|██████████| 50000/50000 [00:18<00:00, 2661.90it/s]  
Finished [100%]: Average Loss = -294.24


In [ ]:
# MCMC error
x.set_value(sm_test)
ppc = pm.sample_posterior_predictive(trace, model=linreg_model, samples=1000)
pred_mcmc = ppc['Y'].mean(axis=0)
mse_mcmc = mean_squared_error(lm_test, pred_mcmc)
print(mse_mcmc)

In [19]:
# ADVI error
x.set_value(sm_test)
ppc = pm.sample_posterior_predictive(approx.sample(100000), model=linreg_model, samples=1000)
pred_advi = ppc['Y'].mean(axis=0)
mse_advi = mean_squared_error(lm_test, pred_advi)
print(mse_advi)

100%|██████████| 1000/1000 [00:13<00:00, 72.29it/s]


0.0025137717692478565


In [ ]:
# There is unused code below, it doesn't work yet

In [ ]:
test_mse = tt.sqr(pred_sample - lm_test).mean(-1)
# train_mse = tt.sqr(train_metrics - lm_train).mean(-1)
eval_tracker = pm.callbacks.Tracker(
    test_mse=test_mse.eval,
)

inference.fit(40, callbacks=[eval_tracker])

In [ ]:
sampled = np.asarray(eval_tracker['samples'])
print(sampled[0])
print(x_0.eval())

In [ ]:
x_0 = theano.shared(sm_train[:, 0])
x_1 = theano.shared(sm_train[:, 1])
x_2 = theano.shared(sm_train[:, 2])
y = theano.shared(lm_train)

model = pm.Model()
with model:
    intercept_2 = pm.Normal('Intercept_2', mu=0, sigma=1)
    coef_2 = pm.Normal('Coef_2', mu=0, sigma=1, shape=2)
    
    xm_2 = pm.Normal('Xm_2', mu=intercept_2 + x_0 * coef_2[0]  + x_1 * coef_2[1], sigma=0.2, observed=x_2)
    coef_y = pm.Normal('Coef_y', mu=0, sigma=1)
    intercept_y = pm.Normal('Intercept_y', mu=0, sigma=1)
    y = pm.Normal('Y', mu=intercept_y + xm_2*coef_y, sigma=0.2, observed=y)


In [ ]:
with model:
    
    inference = pm.SVGD(n_particles=500, jitter=1)

    approx = inference.approx

    replacements = {
                    x_0: sm_test[:, 0],
                    x_1: sm_test[:, 1],
                    x_2: sm_test[:, 2],
                    }

    test_intercept_2 = approx.sample_node(intercept_2, more_replacements=replacements, size=10)
    test_coef_2 = approx.sample_node(coef_2, more_replacements=replacements, size=10)
    test_intercept_y = approx.sample_node(intercept_y, more_replacements=replacements, size=10)
    test_coef_y = approx.sample_node(coef_y, more_replacements=replacements, size=10)


In [ ]:
x_0 = theano.shared(sm_test[:, 0])
x_1 = theano.shared(sm_test[:, 1])
x_2 = theano.shared(sm_test[:, 2])
y = theano.shared(lm_test)

test_mse = tt.sqr(x_2 * test_coef_y - y).mean(-1)
# train_mse = tt.sqr(train_metrics - lm_train).mean(-1)
eval_tracker = pm.callbacks.Tracker(
    test_accuracy=test_mse.eval,
    train_accuracy=train_mse.eval
)

inference.fit(40, callbacks=[eval_tracker])

In [ ]:
eval_tracker = pm.callbacks.Tracker(
    test_mse=test_metrics.eval,
)

inference.fit(40, callbacks=[eval_tracker])

In [ ]:
mse = np.array(eval_tracker['test_mse'])
print(mse.shape)

In [ ]:
print(test_metrics.eval().shape)
# test_data = pd.DataFrame(np.asarray(eval_tracker['test_accuracy']).T).melt()
# print(test_data)

In [ ]:
test_data = pd.DataFrame(np.asarray(eval_tracker['test_accuracy']).T).melt()

# plt.plot(np.asarray(eval_tracker['train_accuracy']), color='blue')
sns.lineplot(x="variable", y="value", data=test_data, color='red')# sns.lineplot(data=np.asarray(eval_tracker['train_accuracy']).T, color='blue')
plt.legend(['test_accuracy', 'train_accuracy'])
plt.title('Training Progress')